In [57]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기

In [58]:
filepath = '/content/drive/Shareddrives/팔당댐/combined_data/combined_train_test_filled_cleansed.csv'
df = pd.read_csv(filepath, index_col='ymdhm', parse_dates=['ymdhm'])
df

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
ymdhm,,,,,,,,,,,,,,,,,
2012-05-01 00:00:00,24.800,555.00,219.07,24.93,555.00,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:10:00,24.794,464.60,218.86,25.15,562.90,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:20:00,24.789,478.10,218.69,25.31,576.40,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:30:00,24.789,464.80,218.69,25.31,563.10,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17,0.0,0.0,0.0
2012-05-01 00:40:00,24.789,478.10,218.69,25.31,576.40,450.0,311.7,476.21,301.2,0.0,291.0,707.17,277.3,564.29,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-18 23:10:00,25.040,259.23,212.86,31.14,259.23,510.0,288.0,319.84,295.0,0.0,299.0,-456.41,306.0,974.40,0.0,0.0,0.0
2022-07-18 23:20:00,25.040,260.46,212.86,31.14,260.46,492.0,287.0,314.01,303.0,0.0,304.0,-717.30,308.0,1006.88,0.0,0.0,0.0
2022-07-18 23:30:00,25.040,259.37,212.86,31.14,259.37,475.0,299.0,387.55,307.0,0.0,309.0,-843.37,310.0,1039.90,0.0,0.0,0.0


In [59]:
df.describe()

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
count,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000,276336.0,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000,276336.000000
mean,25.079251,548.548083,219.425047,24.575537,542.036770,345.631700,329.232797,746.826551,316.071760,0.0,312.590404,600.970021,299.979812,1056.807758,0.034074,0.038265,0.035985
std,0.146126,1110.978412,9.912109,9.911699,1104.317791,174.250734,76.559900,1077.388189,67.495669,0.0,63.883452,1376.848129,51.442783,1428.279212,0.338607,0.370553,0.360769
min,23.993000,-807.500000,176.380000,2.150000,-222.000000,39.000000,259.700000,243.290000,246.200000,0.0,247.000000,-3522.710000,241.300000,212.040000,0.000000,0.000000,0.000000
25%,24.980000,127.417500,211.770000,14.670000,132.000000,192.000000,287.700000,319.840000,278.200000,0.0,275.000000,205.082500,267.300000,449.120000,0.000000,0.000000,0.000000
50%,25.100000,249.570000,217.630000,26.370000,174.980000,324.000000,307.000000,441.000000,297.200000,0.0,295.000000,355.980000,286.300000,679.420000,0.000000,0.000000,0.000000
75%,25.190000,530.000000,229.330000,32.230000,505.000000,487.000000,342.700000,725.690000,330.200000,0.0,328.000000,709.710000,316.300000,1142.220000,0.000000,0.000000,0.000000
max,25.449000,25991.500000,241.850000,67.620000,25791.500000,842.000000,1287.700000,9405.640000,1146.200000,0.0,1080.000000,25289.170000,843.300000,29854.120000,24.000000,21.000000,24.000000


In [60]:
df.isnull().sum()

swl            0
inf            0
sfw            0
ecpc           0
tototf         0
tide_level     0
wl_1018662     0
fw_1018662     0
wl_1018680     0
fw_1018680     0
wl_1018683     0
fw_1018683     0
wl_1019630     0
fw_1019630     0
rf_10184100    0
rf_10184110    0
rf_10184140    0
dtype: int64

# data shift 하고 train, test 데이터 나누기

In [61]:
sub = pd.read_csv('/content/drive/Shareddrives/팔당댐/competition_data/sample_submission.csv',index_col=0)

In [62]:
sub.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,0,0,0,0
2022-06-01 00:10,0,0,0,0
2022-06-01 00:20,0,0,0,0
2022-06-01 00:30,0,0,0,0
2022-06-01 00:40,0,0,0,0


In [63]:
df.sort_index(inplace=True)
sub.sort_index(inplace=True)

In [64]:
label = df.loc[:,sub.columns]
data = df.drop(sub.columns,axis=1)

In [65]:
label

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2012-05-01 00:00:00,310.7,300.2,290.0,275.3
2012-05-01 00:10:00,314.7,300.2,290.0,275.3
2012-05-01 00:20:00,313.7,301.2,290.0,275.3
2012-05-01 00:30:00,311.7,301.2,290.0,276.3
2012-05-01 00:40:00,311.7,301.2,291.0,277.3
...,...,...,...,...
2022-07-18 23:10:00,288.0,295.0,299.0,306.0
2022-07-18 23:20:00,287.0,303.0,304.0,308.0
2022-07-18 23:30:00,299.0,307.0,309.0,310.0


In [66]:
data

,swl,inf,sfw,ecpc,tototf,tide_level,fw_1018662,fw_1018680,fw_1018683,fw_1019630,rf_10184100,rf_10184110,rf_10184140
ymdhm,,,,,,,,,,,,,
2012-05-01 00:00:00,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0.0,0.0,0.0
2012-05-01 00:10:00,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,0.0,0.0,0.0
2012-05-01 00:20:00,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,0.0,0.0,0.0
2012-05-01 00:30:00,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,0.0,0.0,0.0
2012-05-01 00:40:00,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-18 23:10:00,25.040,259.23,212.86,31.14,259.23,510.0,319.84,0.0,-456.41,974.40,0.0,0.0,0.0
2022-07-18 23:20:00,25.040,260.46,212.86,31.14,260.46,492.0,314.01,0.0,-717.30,1006.88,0.0,0.0,0.0
2022-07-18 23:30:00,25.040,259.37,212.86,31.14,259.37,475.0,387.55,0.0,-843.37,1039.90,0.0,0.0,0.0


In [67]:
# data와 label 하나 밀어주기 (과거데이터를 사용해야 함으로)
y = label.reset_index(drop=True)
X = data.reset_index(drop=True)

X.index += 1

tot=pd.concat((X,y),axis=1)
tot=tot.sort_index()

tot=tot.iloc[1:-1]

label = tot.loc[:,sub.columns]
data = tot.drop(sub.columns,axis=1)

In [68]:
train_label=label.iloc[:-len(sub),:]
test_label=label.iloc[-len(sub):,:]

train_data=data.iloc[:-len(sub),:]
test_data=data.iloc[-len(sub):,:]

In [69]:
train_data # ~2022-05-31까지

,swl,inf,sfw,ecpc,tototf,tide_level,fw_1018662,fw_1018680,fw_1018683,fw_1019630,rf_10184100,rf_10184110,rf_10184140
1,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0.0,0.0,0.0
2,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,0.0,0.0,0.0
3,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,0.0,0.0,0.0
4,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,0.0,0.0,0.0
5,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269419,25.180,140.89,217.99,26.01,140.89,232.0,314.01,0.0,597.98,471.08,0.0,0.0,0.0
269420,25.180,140.94,217.99,26.01,140.94,220.0,285.72,0.0,575.57,493.58,0.0,0.0,0.0
269421,25.180,141.07,217.99,26.01,141.07,208.0,274.78,0.0,501.04,505.03,0.0,0.0,0.0
269422,25.180,141.01,217.99,26.01,141.01,196.0,269.40,0.0,425.89,505.03,0.0,0.0,0.0


In [70]:
test_data # 2022-06-01 ~

,swl,inf,sfw,ecpc,tototf,tide_level,fw_1018662,fw_1018680,fw_1018683,fw_1019630,rf_10184100,rf_10184110,rf_10184140
269424,25.19,140.56,218.36,25.64,140.56,173.0,269.40,0.0,70.09,482.26,0.0,0.0,0.0
269425,25.19,140.60,218.36,25.64,140.60,162.0,269.40,0.0,8.23,471.08,0.0,0.0,0.0
269426,25.19,140.78,218.36,25.64,140.78,151.0,280.22,0.0,28.82,449.12,0.0,0.0,0.0
269427,25.20,755.90,218.73,25.27,140.90,141.0,296.87,0.0,12.35,417.17,0.0,0.0,0.0
269428,25.19,0.00,218.36,25.64,140.94,130.0,302.53,0.0,53.52,386.43,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,25.04,865.96,212.86,31.14,259.30,528.0,319.84,0.0,25.27,926.70,0.0,0.0,0.0
276332,25.04,259.23,212.86,31.14,259.23,510.0,319.84,0.0,-456.41,974.40,0.0,0.0,0.0
276333,25.04,260.46,212.86,31.14,260.46,492.0,314.01,0.0,-717.30,1006.88,0.0,0.0,0.0
276334,25.04,259.37,212.86,31.14,259.37,475.0,387.55,0.0,-843.37,1039.90,0.0,0.0,0.0


In [71]:
# scaler = StandardScaler()
# minmax = MinMaxScaler()

In [72]:
# x_train_std = scaler.fit_transform(train_data) # 학습데이터 feature scaling
# x_test_std = scaler.transform(test_data)

In [73]:
# x_train_minmax = minmax.fit_transform(train_data) # 학습데이터 feature scaling
# x_test_minmax = minmax.transform(test_data)

In [74]:
# train_data = x_train_minmax
# test_data = x_test_minmax

In [75]:
# train_data = x_train_std
# test_data = x_test_std

# train, test 데이터 나누기 (X)
- 2022-05-31 까지 train
- 2022-06-01 부터 test 데이터로 사용


In [76]:
#train_data = df.loc[:'2022-05-31',
#                 ['swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level', 'fw_1018662', 'fw_1018680', 
#                  'fw_1018683','fw_1019630', 'rf_10184100', 'rf_10184110', 'rf_10184140']]
#train_label = df.loc[:'2022-05-31',['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']]

# X_train, y_train

In [77]:
#train_data

In [78]:
#test_data = df.loc['2022-06-01':,  
#                   ['swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level', 'fw_1018662', 'fw_1018680', 
#                   'fw_1018683','fw_1019630', 'rf_10184100', 'rf_10184110', 'rf_10184140']]
#test_label = df.loc['2022-06-01': , ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']]

# X_test, y_test

In [79]:
#test_data

# 하이퍼 파라미터

In [80]:
knn = KNeighborsRegressor()
param_grid = [{'n_neighbors': range(3,30)}]
grid_cv = GridSearchCV(estimator=knn, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)
grid_cv.fit(train_data, train_label)

print('최적 하이퍼 파라미터:', grid_cv.best_params_)
print('최적 예측 정확도:', grid_cv.best_score_)

최적 하이퍼 파라미터: {'n_neighbors': 17}
최적 예측 정확도: 0.9879786375375094


최적 하이퍼 파라미터: {'n_neighbors': 19}
최적 예측 정확도: 0.9886472774751205

최적 하이퍼 파라미터: {'n_neighbors': 18}
최적 예측 정확도: 0.9886520136126551

# 모델 학습

In [81]:
model = grid_cv.best_estimator_

In [82]:
kn = KNeighborsRegressor(n_neighbors= 17)
kn.fit(train_data, train_label)

KNeighborsRegressor(n_neighbors=17)

# 모델 성능

In [83]:
# train rmse
train_predict = kn.predict(train_data)
print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, train_label))))
 
# test rmse
y_preds_rnd = kn.predict(test_data)
print("RMSE':{}".format(math.sqrt(mean_squared_error(test_label, y_preds_rnd))))

RMSE':4.005731038000169
RMSE':4.964952049678677


In [84]:
r2_score_train = model.score(train_data, train_label)
print('훈련세트에서의 R^2 점수 : {0:2f}'.format(r2_score_train))

훈련세트에서의 R^2 점수 : 0.996266


In [85]:
r2_score = model.score(test_data, test_label)
print('테스트세트에서의 R^2 점수 : {0:2f}'.format(r2_score))

테스트세트에서의 R^2 점수 : 0.996154


# 테스트세트 예측결과 샘플 출력


In [ ]:
test_label['wl_1018662']
test_label['wl_1018680']
test_label['wl_1018683']
test_label['wl_1019630']


ymdhm
2022-06-01 00:00:00    269.0
2022-06-01 00:10:00    267.0
2022-06-01 00:20:00    264.0
2022-06-01 00:30:00    261.0
2022-06-01 00:40:00    259.0
                       ...  
2022-07-18 23:10:00    306.0
2022-07-18 23:20:00    308.0
2022-07-18 23:30:00    310.0
2022-07-18 23:40:00    312.0
2022-07-18 23:50:00    313.0
Name: wl_1019630, Length: 6912, dtype: float64

In [ ]:
test_label['wl_1018662'][0]


279.0

# 예측

In [86]:
pred_df = pd.DataFrame(y_preds_rnd)
pred_df.head()

,0,1,2,3
0,281.288235,271.552941,271.117647,267.711765
1,281.641176,272.317647,272.411765,266.594118
2,282.935294,270.729412,270.117647,264.652941
3,283.052941,278.670588,272.882353,260.123529
4,283.817647,270.435294,269.588235,259.947059


In [87]:
pred_df.shape

(6912, 4)

# 제출양식

In [36]:
sub_df = pd.read_csv('/content/drive/Shareddrives/팔당댐/competition_data/sample_submission.csv',index_col=0)
sub_df.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,0,0,0,0
2022-06-01 00:10,0,0,0,0
2022-06-01 00:20,0,0,0,0
2022-06-01 00:30,0,0,0,0
2022-06-01 00:40,0,0,0,0


In [88]:
sub_df.shape

(6912, 4)

In [89]:
sub_df['wl_1018662'] = pred_df[0]
sub_df['wl_1018680'] = pred_df[1]
sub_df['wl_1018683'] = pred_df[2]
sub_df['wl_1019630'] = pred_df[3]

In [90]:
sub_df.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,NaN,NaN,NaN,NaN
2022-06-01 00:10,NaN,NaN,NaN,NaN
2022-06-01 00:20,NaN,NaN,NaN,NaN
2022-06-01 00:30,NaN,NaN,NaN,NaN
2022-06-01 00:40,NaN,NaN,NaN,NaN


In [91]:
sub_df = sub_df.astype(float)

In [92]:
pred_df.columns = sub_df.columns

In [93]:
pred_df.index = sub_df.index

In [94]:
pred_df.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,281.288235,271.552941,271.117647,267.711765
2022-06-01 00:10,281.641176,272.317647,272.411765,266.594118
2022-06-01 00:20,282.935294,270.729412,270.117647,264.652941
2022-06-01 00:30,283.052941,278.670588,272.882353,260.123529
2022-06-01 00:40,283.817647,270.435294,269.588235,259.947059


In [ ]:
pred_df.to_csv('/content/drive/Shareddrives/팔당댐/제출/KNN_Regression.csv')

In [95]:
pred_df.to_csv('/content/drive/MyDrive/팔당댐/KNN_Regression17.csv')

# 비교

In [107]:
filename = '/content/drive/Shareddrives/팔당댐/combined_data/combined_train_test_filled.csv'
dff = pd.read_csv(filename, index_col='ymdhm', parse_dates=['ymdhm'])
dff.tail()

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
ymdhm,,,,,,,,,,,,,,,,,
2022-07-18 23:10:00,25.04,259.23,212.86,31.14,259.23,510.0,288.0,319.84,295.0,NaN,299.0,-456.41,306.0,974.40,0.0,0.0,0.0
2022-07-18 23:20:00,25.04,260.46,212.86,31.14,260.46,492.0,287.0,314.01,303.0,NaN,304.0,-717.30,308.0,1006.88,0.0,0.0,0.0
2022-07-18 23:30:00,25.04,259.37,212.86,31.14,259.37,475.0,299.0,387.55,307.0,NaN,309.0,-843.37,310.0,1039.90,0.0,0.0,0.0
2022-07-18 23:40:00,25.04,259.13,212.86,31.14,259.13,458.0,309.0,454.91,311.0,NaN,314.0,-1023.37,312.0,1073.46,0.0,0.0,0.0
2022-07-18 23:50:00,25.04,258.16,212.86,31.14,258.16,442.0,326.0,582.21,314.0,NaN,317.0,-1049.44,313.0,1090.45,0.0,0.0,0.0


In [108]:
dff.loc['2022-06-01': , ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']].head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00:00,279.0,276.0,274.0,269.0
2022-06-01 00:10:00,281.0,277.0,274.0,267.0
2022-06-01 00:20:00,284.0,277.0,274.0,264.0
2022-06-01 00:30:00,285.0,277.0,274.0,261.0
2022-06-01 00:40:00,284.0,278.0,274.0,259.0


In [109]:
pred_df.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,281.288235,271.552941,271.117647,267.711765
2022-06-01 00:10,281.641176,272.317647,272.411765,266.594118
2022-06-01 00:20,282.935294,270.729412,270.117647,264.652941
2022-06-01 00:30,283.052941,278.670588,272.882353,260.123529
2022-06-01 00:40,283.817647,270.435294,269.588235,259.947059


In [ ]:
pred_df.head()

,wl_1018662,wl_1018680,wl_1018683,wl_1019630
ymdhm,,,,
2022-06-01 00:00,280.331579,271.463158,271.947368,268.457895
2022-06-01 00:10,281.278947,270.515789,270.157895,267.142105
2022-06-01 00:20,282.331579,277.936842,272.421053,261.563158
2022-06-01 00:30,282.963158,270.831579,270.157895,261.615789
2022-06-01 00:40,282.805263,269.568421,268.105263,259.036842
